# CliffordLab Companion Notebook

This notebook accompanies the paper *Unified Slice Regularity on Clifford Algebras* (arXiv submission). It mirrors the minimal examples in the Computational Appendix and adds a few convenience utilities for quick checks.

**Sections**
1. Setup & environment
2. CR residuals on $J^2=\pm 1$ slices
3. Cauchy-type reconstruction hooks
4. Symbolic CR checks (SymPy)
5. Micro-benchmarks

> If you have the `CliffordLab` package locally, this will use it. Otherwise, the notebook will fall back to light-weight stubs so you can still inspect the workflow.


## 1) Setup & environment

In [ ]:

# Basic environment info (for reproducibility)
import sys, platform, math, random, statistics, time
print("Python:", sys.version)
print("Platform:", platform.platform())

# Optional imports. These are used if present.
try:
    import numpy as np
    print("NumPy:", np.__version__)
except Exception as e:
    print("NumPy not available:", e)

try:
    import sympy as sp
    print("SymPy:", sp.__version__)
except Exception as e:
    print("SymPy not available:", e)

# Try to import CliffordLab. If unavailable, define stubs below.
try:
    import cliffordlab as CL
    print("CliffordLab: available")
except Exception as e:
    print("CliffordLab not available:", e)
    CL = None


## 2) CR residuals on $J^2 = \pm 1$
We evaluate generalized Cauchy–Riemann residuals on both elliptic and hyperbolic slices for the slice-exponential $e^{x+yJ}$. With `CliffordLab`, use `CL.CR_residuals` directly.

In [ ]:

# If CliffordLab is missing, create stubs that mimic the expected API.
if CL is None:
    import math
    def slice_exp(z, sign, N=40):
        x, y = z
        # closed-form within a slice CJ (commutative): e^{x+yJ} = e^x * (cos/sin or cosh/sinh)
        if sign == -1:
            A = math.exp(x) * math.cos(y)
            B = math.exp(x) * math.sin(y)
        else:
            A = math.exp(x) * math.cosh(y)
            B = math.exp(x) * math.sinh(y)
        return A, B

    def CR_residuals(fAB, x, y, sign, h=1e-5):
        # finite-difference CR residuals:
        # elliptic: A_x =  B_y, A_y = -B_x
        # hyperbolic: A_x = B_y, A_y =  B_x
        A, B = fAB(x, y, sign)
        Ax = (fAB(x+h,y,sign)[0] - fAB(x-h,y,sign)[0])/(2*h)
        Ay = (fAB(x,y+h,sign)[0] - fAB(x,y-h,sign)[0])/(2*h)
        Bx = (fAB(x+h,y,sign)[1] - fAB(x-h,y,sign)[1])/(2*h)
        By = (fAB(x,y+h,sign)[1] - fAB(x,y-h,sign)[1])/(2*h)
        if sign == -1:
            r1 = Ax - By
            r2 = Ay + Bx
        else:
            r1 = Ax - By
            r2 = Ay - Bx
        return r1, r2

    # expose stubs under a CL-like namespace
    class _CLStub:
        slice_exp = staticmethod(lambda z, sign, N=40: slice_exp(z, sign, N))
        CR_residuals = staticmethod(lambda fAB, x, y, sign: CR_residuals(fAB, x, y, sign))
    CL = _CLStub()

def fAB(x, y, sign):
    # Wrap slice exponential for the residual checker
    return CL.slice_exp((x, y), sign, N=40)

for sign in (-1, +1):
    r1, r2 = CL.CR_residuals(fAB, 0.2, 0.3, sign)
    print(f"sign={sign:+d} residuals:", r1, r2)


## 3) Cauchy-type reconstruction (hooks)
Below are hooks mirroring the API from the appendix. If `CliffordLab` is not installed, we provide a placeholder implementation (no heavy numerics).

In [ ]:

# Hooks for Cauchy-type reconstruction
if hasattr(CL, "cauchy_reconstruct"):
    def cauchy_reconstruct(fAB, z0, sign, radius=0.35, N=800):
        return CL.cauchy_reconstruct(fAB, z0, sign, radius=radius, N=N)
else:
    # Lightweight placeholder that simply returns direct evaluation
    def cauchy_reconstruct(fAB, z0, sign, radius=0.35, N=800):
        x0, y0 = z0
        return fAB(x0, y0, sign)

z0 = (0.2, 0.3)
rec_e = cauchy_reconstruct(fAB, z0, -1, radius=0.35, N=800)
rec_h = cauchy_reconstruct(fAB, z0, +1, radius=0.35, N=800)
print("elliptic reconstruction:", rec_e)
print("hyperbolic reconstruction:", rec_h)


## 4) Symbolic CR checks (SymPy)
Derive the CR identities for the slice exponential in both cases. If SymPy is missing, this cell will skip.

In [ ]:

try:
    import sympy as sp
    x, y = sp.symbols('x y', real=True)
    # Elliptic case (J^2 = -1): A=exp(x)cos(y), B=exp(x)sin(y)
    Ae = sp.exp(x)*sp.cos(y)
    Be = sp.exp(x)*sp.sin(y)
    r1e = sp.simplify(sp.diff(Ae,x) - sp.diff(Be,y))   # should be 0
    r2e = sp.simplify(sp.diff(Ae,y) + sp.diff(Be,x))   # should be 0
    print("Elliptic CR residuals (symbolic):", r1e, r2e)

    # Hyperbolic case (J^2 = +1): A=exp(x)cosh(y), B=exp(x)sinh(y)
    Ah = sp.exp(x)*sp.cosh(y)
    Bh = sp.exp(x)*sp.sinh(y)
    r1h = sp.simplify(sp.diff(Ah,x) - sp.diff(Bh,y))   # should be 0
    r2h = sp.simplify(sp.diff(Ah,y) - sp.diff(Bh,x))   # should be 0
    print("Hyperbolic CR residuals (symbolic):", r1h, r2h)
except Exception as e:
    print("SymPy symbolic check skipped:", e)


## 5) Micro-benchmarks
We provide a small timing harness you can adapt. Replace the placeholder with the true `CL.cauchy_reconstruct` when available.

In [ ]:

import time

def benchmark_slice_types(trials=50, sizes=(100, 200, 400, 800)):
    times_elliptic, times_hyperbolic = [], []
    for n in sizes:
        # Elliptic
        start = time.time()
        for _ in range(trials):
            _ = cauchy_reconstruct(fAB, (0.2, 0.3), -1, N=n)
        times_elliptic.append(time.time() - start)

        # Hyperbolic
        start = time.time()
        for _ in range(trials):
            _ = cauchy_reconstruct(fAB, (0.2, 0.3), +1, N=n)
        times_hyperbolic.append(time.time() - start)
    return sizes, times_elliptic, times_hyperbolic

sizes, te, th = benchmark_slice_types(trials=10, sizes=(50,100,200))
print("sizes:", sizes)
print("elliptic timings:", te)
print("hyperbolic timings:", th)


---
**Notes**
- For full functionality use the companion `CliffordLab` package.
- The hooks here mirror the API shown in the paper’s appendix so you can drop in the real implementations without changing cell logic.
- For arXiv reproducibility, consider exporting this notebook to a static HTML and linking it in your README.